# 호현아 부탁한다 이거 굉장히 오래걸릴거야
- 1,000개당 1분 20초 정도 걸림
- 그걸 254만개에 수행해야 함
- 부탁함

In [7]:
# -*- coding: utf-8 -*-
from collections import namedtuple
import time
from collections import OrderedDict
import sys
import xml.etree.ElementTree as ET
import requests

base_url = 'https://m.search.naver.com/p/csearch/ocontent/spellchecker.nhn'

_agent = requests.Session()
PY3 = sys.version_info[0] == 3

class CheckResult:
    PASSED = 0
    WRONG_SPELLING = 1
    WRONG_SPACING = 2
    AMBIGUOUS = 3

def _remove_tags(text):
    text = u'<content>{}</content>'.format(text).replace('<br>','')
    if not PY3:
        text = text.encode('utf-8')
    result = ''.join(ET.fromstring(text).itertext())
    return result

# 조사와 어미도 단어로 처리함. 마땅한 영단어가 생각이 안 나서..
_checked = namedtuple('Checked',
    ['result', 'original', 'checked', 'errors', 'words', 'time'])

class Checked(_checked):
    def __new__(cls, result=False, original='', checked='', 
                errors=0, words=[], time=0.0):
        return super(Checked, cls).__new__(
            cls, result, original, checked, errors, words, time)

    def as_dict(self):
        d = {
            'result': self.result,
            'original': self.original,
            'checked': self.checked,
            'errors': self.errors,
            'words': self.words,
            'time': self.time,
        }
        return d
    
def spallcheck(text):
    if isinstance(text, list):
        result = []
        for item in text:
            checked = spallcheck(item)
            result.append(checked)
        return result
    
    if len(text) > 500:
        return Checked(result=False)
    
    payload = {
        '_callback':'window.__jindo2_callback._spellingCheck_0',
        'q': text
    }
    headers = {
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'referer': 'https://search.naver.com/'
    }
    
    start_time = time.time()
    r = _agent.get(base_url, params=payload, headers=headers)
    passed_time = time.time() - start_time
    
    r = r.text[42:-2]
    
    data = json.loads(r)
    html = data['message']['result']['html']
    result = {
        'result': True,
        'original': text,
        'checked': _remove_tags(html),
        'errors': data['message']['result']['errata_count'],
        'time': passed_time,
        'words': OrderedDict(),
    }
    
    # 띄어쓰기로 구분하기 위해 태그는 일단 보기 쉽게 바꿔둠.
    # ElementTree의 iter()를 써서 더 좋게 할 수 있는 방법이 있지만
    # 이 짧은 코드에 굳이 그렇게 할 필요성이 없으므로 일단 문자열을 치환하는 방법으로 작성.
    html = html.replace('<span class=\'re_green\'>', '<green>') \
               .replace('<span class=\'re_red\'>', '<red>') \
               .replace('<span class=\'re_purple\'>', '<purple>') \
               .replace('</span>', '<end>')
    items = html.split(' ')
    words = []
    tmp = ''
    for word in items:
        if tmp == '' and word[:1] == '<':
            pos = word.find('>') + 1
            tmp = word[:pos]
        elif tmp != '':
            word = u'{}{}'.format(tmp, word)

        if word[-5:] == '<end>':
            word = word.replace('<end>', '')
            tmp = ''

        words.append(word)
        
    for word in words:
        check_result = CheckResult.PASSED
        if word[:5] == '<red>':
            check_result = CheckResult.WRONG_SPELLING
            word = word.replace('<red>', '')
        elif word[:7] == '<green>':
            check_result = CheckResult.WRONG_SPACING
            word = word.replace('<green>', '')
        elif word[:8] == '<purple>':
            check_result = CheckResult.AMBIGUOUS
            word = word.replace('<purple>', '')

        result['words'][word] = check_result

    result = Checked(**result)
    
    return result

In [1]:
# https://frhyme.github.io/python-lib/pipeline_in_python/

import json
import pandas as pd
import numpy as np
import re
from chatspace import ChatSpace # Requirements;
                                # https://lsjsj92.tistory.com/494
                                # conda install pytorch torchvision cudatoolkit=9.0 -c python
                                # pip install chatspace
from konlpy.tag import Okt
import functools
spacer = ChatSpace()
okt = Okt()

stopwords = list('의가이은을로들는좀잘걍과도를자에그와한것다') + ['으로', '하다', '에서', '이다']

def composeFuncs(func1, func2):
    return lambda x : func2(func1(x))

pipes = [lambda x : re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣]', ' ', x),
         lambda x : spallcheck(x).checked,
         lambda x : x.replace(' ', ''),
         lambda x : spacer.space(x),
         lambda x : okt.morphs(x, stem=True, norm=True),
         lambda x : [s for s in x if s not in stopwords]]

preprocessText = functools.reduce(composeFuncs, pipes)

Loading JIT Compiled ChatSpace Model


C:\ProgramData\Anaconda3\envs\keras\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
reviews5over = pd.read_csv('reviews5over.csv')

In [3]:
reviews5over.head()

,reviews
0,전체 관람가는 아닌 것 같아요
1,디렉터스 컷으로 봐서 거의 3시간 짜리인데 참 흡인력 있다
2,태어나 처음으로 가슴 아리는 영화였다. 20년 이상 지났지만.. 생각하면 또 가슴이...
3,어린시절 고딩 때 봤던 때랑 또 결혼하고 나서 봤을 때의 느낌은 확실히 다르네요. ...
4,토토에게 넓은 세상을 보여주고 픈 알프레도.. 그가 토토를 위해 정을 떼려고 했던 ...


In [4]:
reviews5over.size

2468027

In [17]:
reviews5over['reviews'].values[19215]

'노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡'

In [19]:
preprocessText('노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡 노잼 하핡')

['나',
 '잼하핡',
 '나',
 '잼하핡',
 '나',
 '잼하핡',
 '나',
 '잼하핡',
 '나',
 '잼하핡',
 '나',
 '잼하핡',
 '나',
 '잼하핡']

In [ ]:
import pickle
import time
res = []
start, semi_start = time.time(), time.time()
error_ind = []
for ix, text in enumerate(reviews5over['reviews'].values):
    if ix < 19215: continue
    # 1,000개마다 결과 출력
    if ix % 1000 == 0:
        if time.time() - start >= 60:
            a = (time.time() - start) / 3600
            b = 'hours'
        else:
            a = (time.time() - start) / 60
            b = 'mins'
        print(ix, '\t\tcollapse {:.2f} mins\t\tcumulative time is {:.2f} {:s}'.format(
            (time.time() - semi_start) / 60, a, b))
        semi_start = time.time()
    try:
        res.append(preprocessText(text))
    except:
        print(ix)
        res.append(text)
        error_ind.append(ix)
    # 100,000개마다 결과 저장
    if ix % 100000 == 0:
        with open('preprocessReview_{}'.format(ix), 'wb') as f:
            pickle.dump(res, f, protocol=pickle.HIGHEST_PROTOCOL)
        res = []

20000 		collapse 0.74 mins		cumulative time is 0.74 mins
21000 		collapse 1.05 mins		cumulative time is 0.03 hours
22000 		collapse 1.05 mins		cumulative time is 0.05 hours
23000 		collapse 1.00 mins		cumulative time is 0.06 hours
24000 		collapse 0.98 mins		cumulative time is 0.08 hours
25000 		collapse 0.93 mins		cumulative time is 0.10 hours
26000 		collapse 1.10 mins		cumulative time is 0.11 hours
27000 		collapse 0.97 mins		cumulative time is 0.13 hours
28000 		collapse 0.91 mins		cumulative time is 0.15 hours
29000 		collapse 0.95 mins		cumulative time is 0.16 hours
30000 		collapse 1.18 mins		cumulative time is 0.18 hours
31000 		collapse 1.12 mins		cumulative time is 0.20 hours
32000 		collapse 1.10 mins		cumulative time is 0.22 hours
33000 		collapse 0.96 mins		cumulative time is 0.23 hours
34000 		collapse 0.92 mins		cumulative time is 0.25 hours
35000 		collapse 0.93 mins		cumulative time is 0.26 hours
36000 		collapse 1.16 mins		cumulative time is 0.28 hours
37000 		collaps

160000 		collapse 0.94 mins		cumulative time is 2.38 hours
161000 		collapse 0.91 mins		cumulative time is 2.39 hours
162000 		collapse 0.82 mins		cumulative time is 2.41 hours
163000 		collapse 0.84 mins		cumulative time is 2.42 hours
164000 		collapse 0.80 mins		cumulative time is 2.43 hours
165000 		collapse 0.89 mins		cumulative time is 2.45 hours
166000 		collapse 0.96 mins		cumulative time is 2.46 hours
167000 		collapse 0.95 mins		cumulative time is 2.48 hours
168000 		collapse 0.90 mins		cumulative time is 2.50 hours
169000 		collapse 0.92 mins		cumulative time is 2.51 hours
170000 		collapse 0.89 mins		cumulative time is 2.53 hours
171000 		collapse 0.81 mins		cumulative time is 2.54 hours
172000 		collapse 0.84 mins		cumulative time is 2.55 hours
173000 		collapse 0.87 mins		cumulative time is 2.57 hours
174000 		collapse 0.82 mins		cumulative time is 2.58 hours
175000 		collapse 0.83 mins		cumulative time is 2.60 hours
176000 		collapse 0.87 mins		cumulative time is 2.61 hou

298000 		collapse 0.89 mins		cumulative time is 4.34 hours
299000 		collapse 0.86 mins		cumulative time is 4.35 hours
300000 		collapse 0.88 mins		cumulative time is 4.37 hours
301000 		collapse 0.91 mins		cumulative time is 4.38 hours
302000 		collapse 0.84 mins		cumulative time is 4.39 hours
303000 		collapse 0.79 mins		cumulative time is 4.41 hours
304000 		collapse 0.87 mins		cumulative time is 4.42 hours
305000 		collapse 0.86 mins		cumulative time is 4.44 hours
306000 		collapse 0.89 mins		cumulative time is 4.45 hours
307000 		collapse 0.77 mins		cumulative time is 4.46 hours
308000 		collapse 0.90 mins		cumulative time is 4.48 hours
309000 		collapse 0.90 mins		cumulative time is 4.49 hours
310000 		collapse 0.85 mins		cumulative time is 4.51 hours
311000 		collapse 0.78 mins		cumulative time is 4.52 hours
312000 		collapse 0.80 mins		cumulative time is 4.53 hours
313000 		collapse 0.84 mins		cumulative time is 4.55 hours
314000 		collapse 0.81 mins		cumulative time is 4.56 hou

339221
339222
339223
339224
339225
339226
339227
339228
339229
339230
339231
339232
339233
339234
339235
339236
339237
339238
339239
339240
339241
339242
339243
339244
339245
339246
339247
339248
339249
339250
339251
339252
339253
339254
339255
339256
339257
339258
339259
339260
339261
339262
339263
339264
339265
339266
339267
339268
339269
339270
339271
339272
339273
339274
339275
339276
339277
339278
339279
339280
339281
339282
339283
339284
339285
339286
339287
339288
339289
339290
339291
339292
339293
339294
339295
339296
339297
339298
339299
339300
339301
339302
339303
339304
339305
339306
339307
339308
339309
339310
339311
339312
339313
339314
339315
339316
339317
339318
339319
339320
339321
339322
339323
339324
339325
339326
339327
339328
339329
339330
339331
339332
339333
339334
339335
339336
339337
339338
339339
339340
339341
339342
339343
339344
339345
339346
339347
339348
339349
339350
339351
339352
339353
339354
339355
339356
339357
339358
339359
339360
339361
339362
339363

In [ ]:
2,540,000